In [9]:
results = {}

In [10]:
from python_dataprepare_visualize.steps import convert_txt_to_csv
# TXT转CSV
input_dir = 'data/origin'
output_dir = 'data/csv'
results["txt_to_csv"] = convert_txt_to_csv(input_dir, output_dir)
if not results["txt_to_csv"]["success"]:
    print(results)

2025-08-17 15:54:38.176 | INFO     | utils.dataprocess.vibration_data_loader:convert_txt_to_csv_batch:260 - 在 data/origin 中找到 48 个txt文件
2025-08-17 15:54:38.177 | INFO     | utils.dataprocess.vibration_data_loader:convert_txt_to_csv_batch:265 - 开始处理文件: data/origin\11.txt
2025-08-17 15:54:38.179 | INFO     | utils.dataprocess.vibration_data_loader:from_txt:45 - 开始加载文件: data/origin\11.txt
2025-08-17 15:54:38.180 | DEBUG    | utils.dataprocess.vibration_data_loader:from_txt:46 - 初始化元数据和数据存储
2025-08-17 15:54:38.182 | DEBUG    | utils.dataprocess.vibration_data_loader:from_txt:54 - 读取文件内容
2025-08-17 15:54:38.206 | DEBUG    | utils.dataprocess.vibration_data_loader:from_txt:59 - 解析元数据
2025-08-17 15:54:38.207 | DEBUG    | utils.dataprocess.vibration_data_loader:from_txt:76 - 检测到列名: ['Time', 'Time Signal']
2025-08-17 15:54:38.208 | DEBUG    | utils.dataprocess.vibration_data_loader:from_txt:87 - 解析单位信息
2025-08-17 15:54:38.209 | DEBUG    | utils.dataprocess.vibration_data_loader:from_txt:99 - 解析

In [11]:
# 起始点选择
from python_dataprepare_visualize.steps import select_start_indices
input_dir = 'data/csv'
output_dir = 'data/csv_起始点选择'
vg_delay = 0.0025

results["start_idx_selection"] = select_start_indices(
    input_dir, output_dir, 
    vg_delay=vg_delay
)
if not results["start_idx_selection"]["success"]:
    print(results)

2025-08-17 15:59:41.738 | INFO     | utils.dataprocess.start_idx_visualized_select:__init__:90 - 找到配对文件: 11.csv <-> 11V.csv
2025-08-17 15:59:41.739 | INFO     | utils.dataprocess.start_idx_visualized_select:__init__:90 - 找到配对文件: 12.csv <-> 12V.csv
2025-08-17 15:59:41.740 | INFO     | utils.dataprocess.start_idx_visualized_select:__init__:90 - 找到配对文件: 13.csv <-> 13V.csv
2025-08-17 15:59:41.742 | INFO     | utils.dataprocess.start_idx_visualized_select:__init__:90 - 找到配对文件: 14.csv <-> 14V.csv
2025-08-17 15:59:41.743 | INFO     | utils.dataprocess.start_idx_visualized_select:__init__:90 - 找到配对文件: 15.csv <-> 15V.csv
2025-08-17 15:59:41.744 | INFO     | utils.dataprocess.start_idx_visualized_select:__init__:90 - 找到配对文件: 16.csv <-> 16V.csv
2025-08-17 15:59:41.746 | INFO     | utils.dataprocess.start_idx_visualized_select:__init__:90 - 找到配对文件: 21.csv <-> 21V.csv
2025-08-17 15:59:41.747 | INFO     | utils.dataprocess.start_idx_visualized_select:__init__:90 - 找到配对文件: 22.csv <-> 22V.csv
2025-08-

In [ ]:
from python_dataprepare_visualize.steps import apply_debiasing
input_dir = 'data/csv_起始点选择'
output_dir = 'data/csv_起始点选择_去偏'
truncate_to_min = True
# 去偏处理+截断
results["debiasing"] = apply_debiasing(
    input_dir, output_dir, truncate_to_min=truncate_to_min
)
if not results["debiasing"]["success"]:
    print(results)

2025-08-17 16:14:08.578 | INFO     | utils.dataprocess.debiasing:debias_csv_folder:88 - 找到 24 个CSV文件
2025-08-17 16:14:08.881 | INFO     | utils.dataprocess.debiasing:find_min_length:66 - 最短文件是 12.csv，行数为 25156
2025-08-17 16:14:08.882 | INFO     | utils.dataprocess.debiasing:debias_csv_folder:94 - 将截断所有文件到 25156 行 (与 12.csv 一致)
2025-08-17 16:14:08.883 | INFO     | utils.dataprocess.debiasing:debias_csv_folder:101 - 处理文件: 11.csv
2025-08-17 16:14:08.897 | INFO     | utils.dataprocess.debiasing:debias_csv_folder:108 - 截断文件 11.csv 从 25242 行到 25156 行
2025-08-17 16:14:08.985 | INFO     | utils.dataprocess.debiasing:debias_csv_folder:118 - 已保存到: data/csv_起始点选择_去偏\11.csv
2025-08-17 16:14:08.986 | INFO     | utils.dataprocess.debiasing:debias_csv_folder:101 - 处理文件: 12.csv
2025-08-17 16:14:09.095 | INFO     | utils.dataprocess.debiasing:debias_csv_folder:118 - 已保存到: data/csv_起始点选择_去偏\12.csv
2025-08-17 16:14:09.096 | INFO     | utils.dataprocess.debiasing:debias_csv_folder:101 - 处理文件: 13.csv
2025-

In [14]:
# CSV转NPZ
from python_dataprepare_visualize.steps import convert_csv_to_npz_file
input_dir = 'data/csv_起始点选择_去偏'
output_file = 'data/data.npz'
rows = 4
cols = 6
results["csv_to_npz"] = convert_csv_to_npz_file(
    input_dir, output_file,
    rows=rows, cols=cols
)
if not results["csv_to_npz"]["success"]:
    print(results)


2025-08-17 16:20:13.591 | INFO     | csv2npz:convert_csv_to_npz:76 - 开始CSV到NPZ转换
2025-08-17 16:20:13.592 | INFO     | csv2npz:convert_csv_to_npz:77 - 输入文件夹: data/csv_起始点选择_去偏
2025-08-17 16:20:13.595 | INFO     | csv2npz:convert_csv_to_npz:78 - 输出文件: data/data.npz
2025-08-17 16:20:13.597 | INFO     | csv2npz:convert_csv_to_npz:79 - 网格尺寸: 4x6
2025-08-17 16:20:13.599 | INFO     | csv2npz:convert_csv_to_npz:80 - 使用所有点: True
2025-08-17 16:20:13.602 | INFO     | csv2npz:convert_csv_to_npz:92 - 找到 24 个CSV文件
2025-08-17 16:20:13.605 | INFO     | csv2npz:convert_csv_to_npz:99 - 开始处理数据并组织为网格结构
2025-08-17 16:20:13.606 | INFO     | utils.visualize.data_processor:__init__:66 - 初始化DataProcessor...
2025-08-17 16:20:13.607 | INFO     | utils.visualize.data_processor:_create_file_grid:73 - 从 data/csv_起始点选择_去偏 创建文件网格...
2025-08-17 16:20:13.610 | INFO     | utils.visualize.data_processor:_create_file_grid:104 - 创建了 4×6 的文件网格
2025-08-17 16:20:13.611 | INFO     | utils.visualize.data_processor:_load_data:10

In [15]:
from python_dataprepare_visualize.steps import convert_npz_to_mat_file
input_file = 'data/data.npz'
output_file = 'data/data.mat'
# NPZ转MAT
results["npz_to_mat"] = convert_npz_to_mat_file(
    input_file, output_file
)

2025-08-17 16:21:03.901 | INFO     | npz_to_mat:convert_npz_to_mat:57 - 开始NPZ到MAT转换
2025-08-17 16:21:03.903 | INFO     | npz_to_mat:convert_npz_to_mat:58 - 输入文件: data/data.npz
2025-08-17 16:21:03.904 | INFO     | npz_to_mat:convert_npz_to_mat:59 - 输出文件: data/data.mat
2025-08-17 16:21:03.906 | INFO     | npz_to_mat:convert_npz_to_mat:60 - 包含元数据: True
2025-08-17 16:21:03.909 | INFO     | npz_to_mat:convert_npz_to_mat:72 - 正在加载NPZ文件: data/data.npz
2025-08-17 16:21:03.933 | INFO     | npz_to_mat:convert_npz_to_mat:88 - 数据形状: 时间点 = 25156, 行 = 4, 列 = 6
2025-08-17 16:21:03.937 | INFO     | npz_to_mat:convert_npz_to_mat:109 - 正在保存MAT文件: data/data.mat
2025-08-17 16:21:03.956 | INFO     | npz_to_mat:convert_npz_to_mat:129 - MAT文件转换完成！
2025-08-17 16:21:03.957 | INFO     | npz_to_mat:convert_npz_to_mat:130 - 输出文件: data/data.mat
2025-08-17 16:21:03.960 | INFO     | npz_to_mat:convert_npz_to_mat:131 - 文件大小: 5,031,912 bytes
2025-08-17 16:21:03.962 | INFO     | npz_to_mat:convert_npz_to_mat:132 - 包含的变

In [16]:
# 打印结果摘要
print("\n=== Pipeline Results Summary ===")
for step_name, result in results.items():
    status = "✓" if result.get("success", False) else "✗"
    message = result.get("message", "Unknown status")
    print(f"{status} {step_name}: {message}")


=== Pipeline Results Summary ===
✓ txt_to_csv: TXT to CSV conversion completed successfully
✓ start_idx_selection: Start index selection completed successfully
✓ debiasing: Debiasing completed successfully
✓ csv_to_npz: CSV to NPZ conversion completed successfully
✓ npz_to_mat: NPZ to MAT conversion completed successfully
